# Integrate a regression tree model to classify a person's position based on landmark distances
- Collect Training Data: Capture images with known position labels (e.g., "Standing", "Push-up Up", "Push-up Down", "Lying Down").
- Extract Features: Compute landmark distances (e.g., shoulder-to-hip, hip-to-knee, elbow angle, etc.).
- Train a Decision Tree: Use scikit-learn's DecisionTreeClassifier or DecisionTreeRegressor to classify or predict positions.
- Integrate into Real-Time Detection: Use the trained model to classify the current pose.

---
# Embeddings in a CSV file

---

In [1]:
from ACV_Classifier_Fonctions import Embeddings

emb=Embeddings()

base_folder = "./training_datasets/push-ups_images" 
embeddings_filename = "pushup_all"

pushup_both_sides_embeddings=emb.generate_embeddings(base_folder,embeddings_filename)

/home/romain.meunier@Digital-Grenoble.local/anaconda3/envs/ACV/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1743677389.659587   29422 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743677389.759754   29477 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce GTX 1650/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743677389.847628   29470 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743677389.908322   29469 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743677389.95

Images processed : 29
Images skipped : 0
CSV pathfile generated :./embeddings/pushup_all_embeddings.csv


In [15]:
from ACV_Classifier_Fonctions import Embeddings

emb=Embeddings()

base_folder = "./training_datasets/push-ups_images" 
embeddings_filename = "pushup_2a_4d"
features=[
    'left_shoulder_elbow_wrist_angle',
    'right_shoulder_elbow_wrist_angle',
    'nose_left_to_shoulder_dist',
    'nose_right_to_shoulder_dist',
    'left_wrist_to_left_shoulder_dist',
    'right_wrist_to_right_shoulder_dist'
    ]

pushup_both_sides_embeddings=emb.generate_embeddings(base_folder,embeddings_filename,features,show_images=False)

I0000 00:00:1743672993.199016   26688 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743672993.267164   26892 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce GTX 1650/PCIe/SSE2
W0000 00:00:1743672993.359980   26885 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743672993.407030   26889 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Images processed : 29
Images skipped : 0
CSV pathfile generated :./embeddings/pushup_2a_4d_embeddings.csv


In [7]:
from ACV_Classifier_Fonctions import Embeddings

emb=Embeddings()

kaggle_path = "tr1gg3rtrash/yoga-posture-dataset"
embeddings_filename = "yoga_all"

pushup_both_sides_embeddings=emb.generate_embeddings_from_kaggle(kaggle_path,embeddings_filename)


I0000 00:00:1743683185.984108   32490 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743683186.082943   37718 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce GTX 1650/PCIe/SSE2
W0000 00:00:1743683186.158120   37711 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743683186.201563   37715 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Path to dataset files: /home/romain.meunier@Digital-Grenoble.local/.cache/kagglehub/datasets/tr1gg3rtrash/yoga-posture-dataset/versions/1


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: cHRM: inconsistent chromaticities
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Images processed : 2756
Images skipped : 2
CSV pathfile generated :./embeddings/yoga_all_embeddings.csv


---
# Classification

---

In [3]:
from ACV_Classifier_Fonctions import Classifier
from sklearn.tree import DecisionTreeClassifier

clf=Classifier()

model=DecisionTreeClassifier()
embeddings_filename='pushup_all_embeddings.csv'

clf.fit_and_save_model(model,embeddings_filename)

DecisionTreeClassifier()

In [16]:
from ACV_Classifier_Fonctions import Classifier
from sklearn.tree import DecisionTreeClassifier

clf=Classifier()

model=DecisionTreeClassifier()
embeddings_filename='pushup_2a_4d_embeddings.csv'

clf.fit_and_save_model(model,embeddings_filename)

DecisionTreeClassifier()

In [ ]:
from ACV_Classifier_Fonctions import Classifier
from sklearn.tree import DecisionTreeClassifier

clf=Classifier()

model=DecisionTreeClassifier()
embeddings_filename='yoga_all_embeddings.csv'

clf.fit_and_save_model(model,embeddings_filename)

---
# Prediction

---

In [ ]:
from ACV_Classifier_Fonctions import Predict

model_filename="pushup_all_DecisionTreeClassifier.pkl"

model=Predict(model_filename)


/home/romain.meunier@Digital-Grenoble.local/anaconda3/envs/ACV/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1743680819.410149   32490 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743680819.509704   32549 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce GTX 1650/PCIe/SSE2


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743680819.598212   32545 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743680819.640452   32541 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [6]:
image_path='./training_datasets/push-ups_images/position_down/image_001.jpg'
image=model.load_image(image_path)
model.predict(image)

'position_down'

In [31]:
import os
base_folder = "./training_datasets/push-ups_images" 

for label in os.listdir(base_folder):
    label_folder = os.path.join(base_folder, label)
    if not os.path.isdir(label_folder):
        continue

    for i,image_name in enumerate(os.listdir(label_folder)):
        image_path = os.path.join(label_folder, image_name)
        print('image : {0}, predict: {1}'.format(i+1,model.load_image_and_predict(image_path)))


image : 1, predict: position_down
image : 2, predict: position_up
image : 3, predict: position_up
image : 4, predict: position_up
image : 5, predict: position_up
image : 6, predict: position_up
image : 7, predict: position_up
image : 8, predict: position_up
image : 9, predict: position_up
image : 10, predict: position_down
image : 11, predict: position_up
image : 12, predict: position_up
image : 13, predict: position_up
image : 14, predict: position_up
image : 15, predict: position_up
image : 16, predict: position_up
image : 17, predict: position_up
image : 1, predict: position_down
image : 2, predict: position_up
image : 3, predict: position_up
image : 4, predict: position_down
image : 5, predict: position_down
image : 6, predict: position_down
image : 7, predict: position_down
image : 8, predict: position_down
image : 9, predict: position_down
image : 10, predict: position_down
image : 11, predict: position_down
image : 12, predict: position_down


In [5]:
from ACV_Classifier_Fonctions import Predict

model_filename="push-up_classifier.pkl"

model=Predict(model_filename)
model.model.feature_names_in_

I0000 00:00:1743677847.068367   29422 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743677847.114007   30364 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce GTX 1650/PCIe/SSE2


array(['left_shoulder_hip_dist', 'left_hip_knee_dist', 'left_elbow_angle',
       'right_shoulder_hip_dist', 'right_hip_knee_dist',
       'right_elbow_angle'], dtype=object)

W0000 00:00:1743677847.200191   30359 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743677847.242331   30358 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
